In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import os
!pip install natsort
from natsort import natsorted 
import cv2
from keras.models import Sequential
from keras.layers.core import Dense, Activation


#### First Order Features

#### Mean : Gives the contribution of individual pixel intensity for the entire image
#### Variance : Used to find how each pixel varies from the neighbouring pixel
#### Standard Deviation : measures the deviation of measured Values or the data from its mean.
#### Skewness : measures of symmetry, or more precisely, the lack of symmetry.
#### Kurtosis : describes the peakedness of e.g. a frequency distribution Second Order Features

#### Contrast : the difference in luminance or colour across the image
#### Energy : It's the rate of change in the color/brightness/magnitude of the pixels over local areas.
#### ASM (Angular second moment) : is a measure of textural Uniformity of an image
#### Entropy : is a statistical measure of randomness that can be used to characterize the texture of the image
#### Homogeneity : homogeneity expresses how similar certain elements (pixels) of the image are.
#### Dissimilarity : is a numerical measure of how different two data objects are.
#### Correlation : Correlation is the process of moving a filter mask often referred to as kernel over the image and computing the sum of products at each location(CNN alike)
#### Coarseness : Describes the roughness/harshness of a textureLabels

#### Class :
#### 1 = Tumor

#### 0 = Not Tumor

In [ ]:
df=pd.read_csv('../input/brain-tumor/Brain Tumor.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Image'] = le.fit_transform(df['Image'])

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier as rf

X = df.drop('Class', axis=1)
y = df['Class']
selector = SelectFromModel(rf(n_estimators=100, random_state=0))
selector.fit(X, y)
support = selector.get_support()
features = X.loc[:,support].columns.tolist()
print(features)
print(rf(n_estimators=100, random_state=0).fit(X,y).feature_importances_)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(rf(n_estimators=100, random_state=0).fit(X,y),random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

In [ ]:
distplot_df=df.loc[:,['Entropy', 'Energy', 'ASM', 'Homogeneity','Kurtosis','Skewness','Correlation','Dissimilarity']]

fig = plt.figure(figsize=(15, 20))
for i, column in enumerate(distplot_df.columns, 1):
    f=plt.subplot(5,2,i)
    sns.distplot(distplot_df[column], color="green",fit_kws={"color":"darkgreen"});

In [ ]:
sns.pairplot(df, hue='Class', size=2.5);


In [ ]:
sns.swarmplot(x=y, y= df['Homogeneity'])
plt.title("Distribution of image Homogenity, by Class");

In [ ]:
sns.catplot(
    data=df, x="Class", y="Energy", hue="Class", kind="violin",
)

In [ ]:
sns.catplot(
    data=df, x="Class", y="Entropy", hue="Class",
    kind="swarm", aspect=.7,)

In [ ]:
with sns.axes_style('white'):
        sns.jointplot("Class", "ASM", df, kind='kde');

In [ ]:
g = sns.catplot(
    data=df,
    x="Class", y="Kurtosis", hue="Class",
    height=5
)
g.set_axis_labels("Snoot length (mm)", "Snoot depth (mm)")

In [ ]:
sns.violinplot(y = df['Skewness'], x = df['Class'])


In [ ]:
sns.jointplot(x = 'Class', y = 'Dissimilarity', data = df,color='purple')

In [ ]:
plt.figure(figsize= (10, 6))
data_pie  = [1683 , 2079]
labels = ["1", "0"]
explode = [0.1, 0]
plt.pie(data_pie ,labels= labels , explode = explode , autopct="%1.2f%%", shadow= True, colors= ['#003f5c', '#bc5090'])
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()  
folder='/kaggle/input/brain-tumor/Brain Tumor/Brain Tumor/'
imgs=[os.path.join(folder,img) for img in os.listdir(folder) if img.endswith('.jpg')]
imgs=natsorted(imgs)

img=cv2.imread(imgs[3760],cv2.IMREAD_GRAYSCALE)

im = plt.imshow(img,  interpolation='none', aspect='auto',cmap ='gray', vmin=0, vmax=255)   
plt.title('No Tumor')

In [ ]:
img=cv2.imread(imgs[3],cv2.IMREAD_GRAYSCALE)

im = plt.imshow(img,  interpolation='none', aspect='auto',cmap ='gray', vmin=0, vmax=255)   
plt.title('Tumor')

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=[2633,14]
        ),
        keras.layers.Dense(515, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

early_stopping = keras.callbacks.EarlyStopping( patience = 20, min_delta = 0.001,
                                               restore_best_weights =True )
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=15,
    epochs=50,
    callbacks = [early_stopping],
    verbose=1, 
)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)


In [ ]:
print('Test accuracy:', round(test_acc,4))


In [ ]:
predictions =(model.predict(X_test)>0.5).astype("int32")
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(y_test, predictions)